In [1]:
using Pkg
using Revise
Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using GLMakie
# Constants and Parameters
const n, a, b = 3, 12, 100 
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = tref_3d # Objective function

  Activating project at `~/globtim`


tref_3d (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
rand_center = [0.0, 0.0, 0,];
d = 14 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.2274670057421434
current L2-norm: 0.2206118118799306


Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [3]:
# df_cheb = solve_and_parse(pol_cheb, x, f, TR)
pts_cheb = solve_polynomial_system(x, TR.dim, d, pol_cheb.coeffs; basis=:chebyshev)
df_cheb = process_critical_points(pts_cheb, f, TR)
sort!(df_cheb, :z, rev=false)

Dimension m of the vector space: 680
Tracking 2197 paths...   0%|                            |  ETA: 1:58:33



Tracking 2197 paths...   7%|█▉                          |  ETA: 0:01:41



Tracking 2197 paths...  10%|██▊                         |  ETA: 0:01:06



Tracking 2197 paths...  13%|███▋                        |  ETA: 0:00:50



Tracking 2197 paths...  16%|████▌                       |  ETA: 0:00:40



Tracking 2197 paths...  19%|█████▍                      |  ETA: 0:00:34



Tracking 2197 paths...  22%|██████▎                     |  ETA: 0:00:28



Tracking 2197 paths...  26%|███████▎                    |  ETA: 0:00:24



Tracking 2197 paths...  28%|███████▉                    |  ETA: 0:00:22



Tracking 2197 paths...  31%|████████▋                   |  ETA: 0:00:20



Tracking 2197 paths...  34%|█████████▍                  |  ETA: 0:00:18



Tracking 2197 paths...  36%|██████████▏                 |  ETA: 0:00:16



Tracking 2197 paths...  38%|██████████▊                 |  ETA:

Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,0.0736422,-0.094467,0.0274242,-2.85823
2,0.0741805,-0.0245244,0.0792558,-2.74385
3,0.074208,0.0647095,0.0803655,-2.72694
4,-0.0212179,-0.0245571,0.0814471,-2.52545
5,-0.0212193,0.0646042,0.0824698,-2.50874
6,0.0740949,-0.0239055,-0.0230697,-2.45652
7,0.0741013,0.0645526,-0.0234259,-2.4548
8,-0.0212524,-0.0944557,0.028739,-2.34301
9,-0.0214254,-0.0238945,-0.0228648,-1.8126


In [4]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [5]:
fig = create_level_set_visualization(f, grid, df_cheb, (-3.0, 6.))

2025-01-30 00:27:54.279 julia[65085:22931685] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-30 00:27:54.279 julia[65085:22931685] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [14]:
using StaticArrays

In [36]:
function video_level_sets(
    f,
    grid::Array{SVector{3,T},3},
    df::DataFrame,
    z_range::Tuple{Real,Real},
    output_path::String;
    params::VisualizationParameters{T}=VisualizationParameters{T}(),
    fps::Int=30,
    duration::Float64=20.0,
    slowdown_factor::Real=10.0
) where {T<:AbstractFloat}

    # Convert z_range to match the float type of the grid
    z_range_converted = (T(z_range[1]), T(z_range[2]))
    z_min, z_max = z_range_converted

    # Get unique z-values from the data points and convert to same type
    data_levels = sort(unique(T.(df[!, "z"])))

    # Create non-uniform level sampling that's denser around data points
    function generate_level_sequence(n_frames)
        base_seq = range(z_min, z_max, length=n_frames)

        function compute_weight(z)
            weight = 1.0
            window = (z_max - z_min) / 10

            for level in data_levels
                dist = abs(z - level)
                if dist < window
                    weight += slowdown_factor * exp(-(dist / window)^2 * 2)
                end
            end
            return weight
        end

        weights = compute_weight.(base_seq)
        cumsum_weights = cumsum(weights)
        normalized_weights = cumsum_weights ./ cumsum_weights[end]

        return [z_min + (z_max - z_min) * w for w in normalized_weights]
    end

    n_frames = round(Int, fps * duration)
    level_sequence = generate_level_sequence(n_frames)

    # Create figure
    fig = Figure(size=params.fig_size)
    ax = Axis3(fig[1, 1],
        title="Level Set Visualization",
        xlabel="x₁",
        ylabel="x₂",
        zlabel="x₃")

    # Extract grid bounds
    grid_points = vec(grid)
    x_range = extrema(p[1] for p in grid_points)
    y_range = extrema(p[2] for p in grid_points)
    z_range_grid = extrema(p[3] for p in grid_points)

    limits!(ax, x_range..., y_range..., z_range_grid...)

    # Pre-calculate values at grid points
    values = reshape(map(f, grid_points), size(grid))

    # Observables for points
    level_points = Observable(Point3f[])
    data_points = Observable(Point3f[])

    # Create visualization elements with smaller markers
    scatter!(ax, level_points,
        color=:blue,
        markersize=1.5,  # Smaller blue points
        label="Level Set")

    scatter!(ax, data_points,
        color=:orange,
        marker=:diamond,
        markersize=15,  # Slightly smaller diamonds
        label="Data Points")

    axislegend(ax, position=:rt)

    function update_visualization(level::T) where {T<:AbstractFloat}
        # Get level set points for current level
        level_data = prepare_level_set_data(grid, values, level, tolerance=params.point_tolerance)
        formatted_data = to_makie_format(level_data)

        # Update level set points
        if !isempty(formatted_data.xyz[1])
            level_points[] = [Point3f(x, y, z) for (x, y, z) in zip(formatted_data.xyz...)]
        else
            level_points[] = Point3f[]  # Clear points if none found
        end

        # Update data points
        visible_points = Point3f[]
        point_tolerance = params.point_tolerance * 1.5
        for row in eachrow(df)
            if abs(T(row["z"]) - level) ≤ point_tolerance
                push!(visible_points, Point3f(row["x1"], row["x2"], row["x3"]))
            end
        end
        data_points[] = visible_points
    end

    # Create and save animation
    record(fig, output_path, 1:n_frames; framerate=fps) do frame
        current_level = level_sequence[frame]
        update_visualization(current_level)

        ax.azimuth[] = 1.7π + 0.4 * sin(2π * frame / n_frames)
        ax.elevation[] = π / 4 + 0.3 * cos(2π * frame / n_frames)
    end

    return nothing
end

video_level_sets (generic function with 2 methods)

In [37]:
# Create visualization parameters
params = VisualizationParameters(
    fig_size=(1920, 1080),  # HD resolution
    point_tolerance=0.01
)

# Generate video
video_level_sets(
    tref_3d,
    grid,
    df_cheb,
    (-1, 1),
    "output.mp4";
    params=params,
    fps=30,  # Still 30 fps
    duration=20.0  # But twice as long
)

In [6]:
# GLMakie.closeall()